# HIV Model using streamlit + Mesa + Agent Based Modeling

## Background
"A traditional framework for infectious disease spread is the so-called SIR model, dividing a population int susceptible (S), infectious (I), and recovered/removed (R). These can be estimated over time with a set of differential equations given known transition rates between states. These in turn depend on parameters like the R0 for the infection. These equation based methods are called compartmental models. Agent-based models are a more recent advance that simulate many individual 'agents' in the population to achieve the same goal. The agents are heterogeneous, with multiple attributes and complexity emerges out of the aggregate behavior of many agents combined." (Farrell, 2020)

## Objective 
To simulate how HIV spreads and evolves in a population over time, and to test how different interventions (like increasing treatment coverage or reducing transmission rates) might affect HIV prevalence, incidence and deaths.<br>

### Shortcomings
Data insufficiency<br>Unfamiliar Knowledge base<br>

### Contingency
Stocks Modeling<br>SEIR/SIR Model Adaptation<br>

### Explanatory Modeling
Explanatory modeling means:<br>
- We observe a phenomenon.
- We have an idea of the conditions that may cause this phenomenon. 
- Our intuition is insufficient as a proof.
- Therefore we observe an approximate representation of the putative causing conditions.
- If the representation show behavior similar to our observation, our initial idea was correct.


In [ ]:
import time
import enum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

class HIVModel(Model):
    """A model for HIV spread and intervention testing."""

    def __init__(self, N=1000, width=20, height=20, 
                 base_transmission_rate=0.04,  # Per contact probability
                 condom_use_rate=0.50,         # Population-level condom usage
                 treatment_coverage=0.20,      # Percentage on effective treatment
                 treatment_effectiveness=0.96, # How much treatment reduces transmission
                 disease_progression_time=3650,  # ~10 years avg from infection to AIDS without treatment
                 disease_progression_sd=1095,   # 3 year standard deviation
                 death_rate_untreated=0.10,    # Annual probability of death for untreated AIDS
                 death_rate_treated=0.02,      # Annual probability of death for treated HIV
                 testing_rate=0.30            # Percentage who get tested per year
                ):

        # Model parameters
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.running = True
        
        # HIV-specific parameters
        self.base_transmission_rate = base_transmission_rate
        self.condom_use_rate = condom_use_rate
        self.treatment_coverage = treatment_coverage
        self.treatment_effectiveness = treatment_effectiveness
        self.disease_progression_time = disease_progression_time
        self.disease_progression_sd = disease_progression_sd
        self.death_rate_untreated = death_rate_untreated / 365  # Convert to daily rate
        self.death_rate_treated = death_rate_treated / 365      # Convert to daily rate
        self.testing_rate = testing_rate / 365                  # Convert to daily rate
        
        # Create agents
        for i in range(self.num_agents):
            a = PersonAgent(i, self)
            self.schedule.add(a)
            
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
            # Make some agents HIV+ at start (1% initial prevalence)
            infected = np.random.choice([0,1], p=[0.99, 0.01])
            if infected == 1:
                a.hiv_status = HIVStatus.INFECTED_UNTREATED
                a.infection_time = 0
                a.progression_time = self.get_progression_time()
        
        # Setup data collection
        self.datacollector = DataCollector(
            model_reporters={
                "Susceptible": lambda m: self.count_status(HIVStatus.SUSCEPTIBLE),
                "HIV+ Untreated": lambda m: self.count_status(HIVStatus.INFECTED_UNTREATED),
                "HIV+ Treated": lambda m: self.count_status(HIVStatus.INFECTED_TREATED),
                "AIDS Untreated": lambda m: self.count_status(HIVStatus.AIDS_UNTREATED),
                "AIDS Treated": lambda m: self.count_status(HIVStatus.AIDS_TREATED),
                "Dead": lambda m: self.count_status(HIVStatus.DEAD),
                "New Infections": lambda m: self.new_infections,
                "Deaths": lambda m: self.deaths_this_step
            },
            agent_reporters={"Status": "hiv_status"})
        
        # Track new infections and deaths
        self.new_infections = 0
        self.deaths_this_step = 0
    
    def count_status(self, status):
        """Helper method to count agents in a specific status"""
        return sum(1 for agent in self.schedule.agents if agent.hiv_status == status)
    
    def get_progression_time(self):
        """Returns time until disease progresses from HIV to AIDS"""
        return int(self.random.normalvariate(self.disease_progression_time, self.disease_progression_sd))

    def step(self):
        """Advance the model by one step"""
        # Reset counters
        self.new_infections = 0
        self.deaths_this_step = 0
        
        # Collect data and advance all agents one step
        self.datacollector.collect(self)
        self.schedule.step()


class HIVStatus(enum.IntEnum):
    """Different states for HIV status"""
    SUSCEPTIBLE = 0
    INFECTED_UNTREATED = 1
    INFECTED_TREATED = 2
    AIDS_UNTREATED = 3
    AIDS_TREATED = 4
    DEAD = 5


class PersonAgent(Agent):
    """An agent representing a person in an HIV model."""
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.age = self.random.normalvariate(30, 15)
        self.hiv_status = HIVStatus.SUSCEPTIBLE
        self.infection_time = None
        self.progression_time = None
        self.knows_status = False
        self.on_treatment = False
        
    def move(self):
        """Move the agent randomly to a neighboring cell"""
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def update_disease_status(self):
        """Update the agent's HIV/AIDS status"""
        # Skip if susceptible or dead
        if (self.hiv_status == HIVStatus.SUSCEPTIBLE or 
            self.hiv_status == HIVStatus.DEAD):
            return
            
        # Get tested with some probability if status unknown
        if not self.knows_status:
            if self.random.random() < self.model.testing_rate:
                self.knows_status = True
                # If they test positive, they might get treatment
                if self.knows_status and self.random.random() < self.model.treatment_coverage:
                    self.on_treatment = True
                    if self.hiv_status == HIVStatus.INFECTED_UNTREATED:
                        self.hiv_status = HIVStatus.INFECTED_TREATED
                    elif self.hiv_status == HIVStatus.AIDS_UNTREATED:
                        self.hiv_status = HIVStatus.AIDS_TREATED
        
        # Determine if disease progresses from HIV to AIDS
        if (self.hiv_status == HIVStatus.INFECTED_UNTREATED or 
            self.hiv_status == HIVStatus.INFECTED_TREATED):
            time_since_infection = self.model.schedule.time - self.infection_time
            
            # Progress more slowly if on treatment (double the time)
            effective_time = time_since_infection
            if self.hiv_status == HIVStatus.INFECTED_TREATED:
                effective_time = time_since_infection / 2
                
            if effective_time >= self.progression_time:
                if self.on_treatment:
                    self.hiv_status = HIVStatus.AIDS_TREATED
                else:
                    self.hiv_status = HIVStatus.AIDS_UNTREATED
        
        # Determine death probability based on status
        death_probability = 0
        if self.hiv_status == HIVStatus.AIDS_UNTREATED:
            death_probability = self.model.death_rate_untreated
        elif self.hiv_status == HIVStatus.AIDS_TREATED:
            death_probability = self.model.death_rate_treated
        elif self.hiv_status == HIVStatus.INFECTED_UNTREATED:
            # Small chance of death even before AIDS
            death_probability = self.model.death_rate_untreated / 10
        
        # Check if the agent dies
        if self.random.random() < death_probability:
            self.hiv_status = HIVStatus.DEAD
            self.model.deaths_this_step += 1
            self.model.schedule.remove(self)

    def interact(self):
        """Potentially transmit HIV to others in the same cell"""
        # Only infectious if HIV+ or AIDS
        if (self.hiv_status != HIVStatus.INFECTED_UNTREATED and 
            self.hiv_status != HIVStatus.INFECTED_TREATED and 
            self.hiv_status != HIVStatus.AIDS_UNTREATED and 
            self.hiv_status != HIVStatus.AIDS_TREATED):
            return
            
        # Find other agents in the same cell
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) <= 1:  # No one else here
            return
            
        # Calculate transmission probability based on status and treatment
        base_prob = self.model.base_transmission_rate
        
        # Higher transmission when viral load is high (early infection or AIDS)
        if self.hiv_status == HIVStatus.AIDS_UNTREATED:
            base_prob *= 2.5  # Higher viral load in AIDS stage
        elif self.hiv_status == HIVStatus.AIDS_TREATED:
            base_prob *= 0.5  # Treatment reduces but doesn't eliminate transmission
            
        # Treatment dramatically reduces transmission
        if self.hiv_status == HIVStatus.INFECTED_TREATED or self.hiv_status == HIVStatus.AIDS_TREATED:
            base_prob *= (1 - self.model.treatment_effectiveness)
            
        # For each susceptible cellmate, calculate transmission
        for other in cellmates:
            if other.hiv_status != HIVStatus.SUSCEPTIBLE:
                continue
                
            # Model a sexual encounter with probability
            if self.random.random() > 0.05:  # 5% chance of interaction per day
                continue
                
            # Check if condom is used (reduces transmission by ~95%)
            condom_used = self.random.random() < self.model.condom_use_rate
            effective_prob = base_prob
            if condom_used:
                effective_prob *= 0.05
                
            # Check if transmission occurs
            if self.random.random() < effective_prob:
                other.hiv_status = HIVStatus.INFECTED_UNTREATED
                other.infection_time = self.model.schedule.time
                other.progression_time = self.model.get_progression_time()
                other.knows_status = False  # Newly infected, doesn't know status
                self.model.new_infections += 1

    def step(self):
        """Advance the agent one step"""
        if self.hiv_status != HIVStatus.DEAD:  # Skip if dead
            self.update_disease_status()
            self.move()
            self.interact()


# Example usage with intervention testing
def run_model_with_params(years=10, population=1000, treatment_coverage=0.2, condom_use=0.5):
    """Run the HIV model with specified parameters"""
    steps = years * 365  # Daily steps for specified years
    model = HIVModel(
        N=population, 
        treatment_coverage=treatment_coverage,
        condom_use_rate=condom_use
    )
    
    for i in range(steps):
        model.step()
        
    return model.datacollector.get_model_vars_dataframe()

# Function to plot results
def plot_hiv_results(results, title='HIV Model Results'):
    """Plot the results of the HIV model simulation"""
    
    # Convert daily steps to years for x-axis
    results = results.reset_index()
    results['Year'] = results['Step'] / 365
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot each status line
    ax.plot(results['Year'], results['Susceptible'], label='Susceptible', color='blue')
    ax.plot(results['Year'], results['HIV+ Untreated'], label='HIV+ Untreated', color='orange')
    ax.plot(results['Year'], results['HIV+ Treated'], label='HIV+ Treated', color='green')
    ax.plot(results['Year'], results['AIDS Untreated'], label='AIDS Untreated', color='red')
    ax.plot(results['Year'], results['AIDS Treated'], label='AIDS Treated', color='purple')
    ax.plot(results['Year'], results['Dead'], label='Cumulative Deaths', color='black')
    
    ax.set_xlabel('Years')
    ax.set_ylabel('Number of People')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Compare different intervention strategies
def compare_interventions():
    """Compare different intervention strategies"""
    # Baseline scenario
    baseline = run_model_with_params(treatment_coverage=0.2, condom_use=0.5)
    
    # High treatment coverage
    high_treatment = run_model_with_params(treatment_coverage=0.9, condom_use=0.5)
    
    # High condom use
    high_condom = run_model_with_params(treatment_coverage=0.2, condom_use=0.9)
    
    # Combined intervention
    combined = run_model_with_params(treatment_coverage=0.9, condom_use=0.9)
    
    # Plot results
    plot_hiv_results(baseline, "Baseline Scenario")
    plot_hiv_results(high_treatment, "High Treatment Coverage (90%)")
    plot_hiv_results(high_condom, "High Condom Use (90%)")
    plot_hiv_results(combined, "Combined Interventions")
    
    # Return the data for further analysis
    return {
        "baseline": baseline,
        "high_treatment": high_treatment,
        "high_condom": high_condom,
        "combined": combined
    }

# To run the simulation and compare interventions:
results = compare_interventions()

# To run a single simulation with specific parameters:
# simulation_data = run_model_with_params(years=20, population=2000, treatment_coverage=0.5, condom_use=0.7)
# plot_hiv_results(simulation_data, "Custom HIV Intervention Scenario")